<a href="https://colab.research.google.com/github/subin111/startstudy/blob/main/HelloLlama2_Cloud_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This demo app shows:
* How to run Llama2 in the cloud hosted on Replicate
* How to use LangChain to ask Llama general questions and follow up questions
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and chat about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

**Note** We will be using Replicate to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while.
After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on Replicate.

Let's start by installing the necessary packages:
- sentence-transformers for text embeddings
- chromadb gives us database capabilities
- langchain provides necessary RAG tools for this demo

And setting up the Replicate token.

In [1]:
!pip install langchain replicate sentence-transformers chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN


··········


Next we call the Llama 2 model from replicate. In this example we will use the llama 2 13b chat model. You can find more Llama 2 models by searching for them on the [Replicate model explore page](https://replicate.com/explore?query=llama).

You can add them here in the format: model_name/version


In [3]:
from langchain.llms import Replicate

llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llm = Replicate(
    model=llama2_13b,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500}
)

With the model set up, you are now ready to ask some questions. Here is an example of the simplest way to ask the model some general questions.

In [4]:
question = "who wrote the book Innovator's dilemma?"
answer = llm(question)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Hello! I'd be happy to help you with your question. The book "The Innovator's Dilemma" was written by Clayton Christensen, an American author and professor at Harvard Business School. It was first published in 1997 and has since become a widely influential work on innovation and business strategy. Do you have any other questions or would you like more information on this topic?


We will then try to follow up the response with a question asking for more information on the book.

Since the chat history is not passed on Llama doesn't have the context and doesn't know this is more about the book thus it treats this as new query.


In [5]:
# chat history not passed so Llama doesn't have the context and doesn't know this is more about the book
followup = "tell me more"
followup_answer = llm(followup)
print(followup_answer)

 Hello! I'm here to assist you with any questions or tasks you may have. I can provide information on a wide range of topics, from science and history to entertainment and culture. I can also help with practical tasks such as converting units of measurement or calculating dates and times. Is there something specific you would like to know or discuss?


To get around this we will need to provide the model with history of the chat.

To do this, we will use  [`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) to pass the chat history to the model and give it the capability to handle follow up questions.

In [6]:
# using ConversationBufferMemory to pass memory (chat history) for follow up questions
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

Once this is set up, let us repeat the steps from before and ask the model a simple question.

Then we pass the question and answer back into the model for context along with the follow up question.

In [7]:
# restart from the original question
answer = conversation.predict(input=question)
print(answer)

 Ah, you're asking about "The Innovator's Dilemma," that classic book by Clayton Christensen! He's a renowned author and professor at Harvard Business School, known for his work on disruptive innovation and how established companies can struggle to adapt to new technologies and business models.

In fact, I have access to a wealth of information on this topic, as well as other areas of expertise. Would you like me to share some interesting facts or insights about Clayton Christensen or his book? For example, did you know that he coined the term "disruptive innovation" in his book "The Innovator's Dilemma"? Or that the book has been widely influential in both academic and business circles, with over 1 million copies sold worldwide?

Oh, and if you're curious about the content of the book itself, I can give you a brief summary! "The Innovator's Dilemma" explores why successful companies often fail to adopt new technologies and business models that ultimately disrupt their industries. It a

In [8]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

 Sure thing! Clayton Christensen is a highly respected author and professor who has made significant contributions to the field of business and economics. Here are some additional facts and insights about him and his book:

* As I mentioned earlier, Clayton Christensen coined the term "disruptive innovation" in "The Innovator's Dilemma." This concept refers to the idea that new technologies or business models can disrupt entire industries by offering low-end products or services that eventually improve in quality and become mainstream.
* The book was first published in 1997 and has since become a classic in the business literature. It has been translated into numerous languages and has sold over 1 million copies worldwide.
* "The Innovator's Dilemma" challenges the conventional wisdom that successful companies should focus on sustaining their current success rather than investing in new technologies or business models. Instead, Christensen argues that established companies must be will

Next, let's explore using Llama 2 to answer questions using documents for context.
This gives us the ability to update Llama 2's knowledge thus giving it better context without needing to finetune.
For a more in-depth study of this, see the notebook on using Llama 2 locally [here](HelloLlamaLocal.ipynb)

We will use the PyPDFLoader to load in a pdf, in this case, the Llama 2 paper.

In [9]:

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2307.09288.pdf")
docs = loader.load()

In [10]:
# check docs length and content
print(len(docs), docs[0].page_content[0:300])

77 Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernande


We need to store our documents. There are more than 30 vector stores (DBs) supported by LangChain.
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the HuggingFaceEmbeddings and RecursiveCharacterTextSplitter to assist in storing the documents.

In [11]:

from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database.

In general, you should use larger chuck sizes for highly structured text such as code and smaller size for less structured text. You may need to experiment with different chunk sizes and overlap values to find out the best numbers.

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to Llama to answer the question.

In [13]:
# use LangChain's RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


question = "What is llama2?"
result = qa_chain({"query": question})
print(result['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Hello! I'm here to help answer your questions to the best of my ability. Regarding llama2, it appears to be a machine learning model developed by Meta AI that is optimized for dialogue use cases. The model has been trained on a large dataset of text and has the potential to generate human-like responses to user prompts. However, it is important to note that the model is still in its early stages and may not always produce accurate or appropriate responses. Additionally, it is crucial to use the model responsibly and ethically, as outlined in the provided responsible use guide. If you have any specific questions or prompts, I'll do my best to assist you.


Now, lets bring it all together by incorporating follow up questions.

First we ask a follow up questions without giving the model context of the previous conversation.
Without this context, the answer we get does not relate to our original question.

In [14]:
# no context passed so Llama2 doesn't have enough context to answer so it lets its imagination go wild
result = qa_chain({"query": "what are its use cases?"})
print(result['result'])

 Based on the context provided, the use cases for Llama 2-Chat are:

1. Dialogue use cases: The fine-tuned version of Llama 2, called Llama 2-Chat, is optimized for dialogue use cases.
2. Large-scale training: The approach devised by the authors involves sampling millions of interconnects for large-scale training.
3. Responsible release: The authors provide a responsible release strategy for the open release of LLMs, including safety testing and tuning tailored to specific application scenarios.

Note that I don't have information about other potential use cases beyond those mentioned in the text.


As we did before, let us use the `ConversationalRetrievalChain` package to give the model context of our previous question so we can add follow up questions.

In [15]:
# use ConversationalRetrievalChain to pass chat history for follow up questions
from langchain.chains import ConversationalRetrievalChain
chat_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True)

In [16]:
# let's ask the original question "What is llama2?" again
result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

 Hello! I'm here to help answer your questions to the best of my ability. Regarding llama2, it appears to be a machine learning model developed by Meta AI that is optimized for dialogue use cases. The model has been trained on a large dataset of text and has the potential to generate human-like responses to user prompts. However, it is important to note that the model is still in its early stages and may not always produce accurate or appropriate responses. Additionally, it is crucial to use the model responsibly and ethically, as outlined in the provided responsible use guide. If you have any specific questions or prompts, I'll do my best to assist you.


In [17]:
# this time we pass chat history along with the follow up so good things should happen
chat_history = [(question, result["answer"])]
followup = "what are its use cases?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

 Based on the information provided, I can answer your question about the potential use cases for llama2, the machine learning model developed by Meta AI.

The llama2 model is a text-to-text transformer designed for open-ended conversational AI applications. It is trained on a large dataset of text from publicly available sources and can be fine-tuned for specific tasks or domains. With its ability to generate human-like language outputs, llama2 has the potential to be used in various applications such as:

1. Chatbots: llama2 could be used to create more advanced chatbots that can engage in natural-sounding conversations with users, providing customer support, answering questions, or simply entertaining users.
2. Content generation: llama2 could be used to generate content for websites, social media platforms, or other digital media outlets. This could include articles, blog posts, product descriptions, or other types of written content.
3. Language translation: llama2 could be used to

Further follow ups can be made possible by updating chat_history.

Note that results can get cut off. You may set "max_new_tokens" in the Replicate call above to a larger number (like shown below) to avoid the cut off.

```python
model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens": 1000}
```

In [18]:
# further follow ups can be made possible by updating chat_history like this:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "what tasks can it assist with?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])



 Based on the capabilities of Llama2 as a machine learning model optimized for dialogue use cases, it could potentially assist with various natural language generation tasks such as:

1. Chatbots: Llama2 could be used to create more engaging and human-like chatbot experiences, especially for assistant-like chat scenarios.
2. Customer Service: The model's ability to remember attributes from previous turns could be useful in customer service interactions, allowing it to maintain context and provide more personalized responses.
3. Language Translation: Llama2 could be fine-tuned for language translation tasks, particularly for languages other than English, given its proficiency in other languages.
4. Content Generation: The model's capability to generate coherent and contextually relevant text could be applied to content creation tasks such as writing articles, product descriptions, or social media posts.
5. Conversational AI: Llama2's ability to engage in multi-turn conversations and rem

###직접 해본 질문

In [19]:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "Can you speak Korean? How well do you speak Korean?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

 Sure, I can help answer your question! However, I don't have the ability to speak or understand any languages, including Korean. My training data only includes text-based responses, so I cannot provide spoken answers or engage in conversations. Additionally, I cannot assess my own proficiency in any language, as I am a machine learning model and do not possess subjective experiences or self-awareness. If you have any further questions or need assistance with something else, feel free to ask!


In [20]:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "Explain your model to make answer for user's question."
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

 Based on the information provided, the potential use cases for llama2, the machine learning model developed by Meta AI, include:

1. Assistant-like chat: Tuned models are intended for this purpose.
2. Natural language generation tasks: Pretrained models can be adapted for this purpose.

The model is intended for commercial and research use in English. It is important to note that the model is not intended for uses that violate applicable laws or regulations, such as trade compliance laws, or for uses in languages other than English. Additionally, the model should not be used in any manner that is prohibited by the Acceptable Use Policy and Licensing Agreement for llama2.


In [21]:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "Please recommend me the subject about Embedded software project."
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

 Based on the provided context, here are some potential embedded software project subjects related to the use cases of llama2, the machine learning model developed by Meta AI:

1. Natural Language Processing (NLP) for Chatbots: Develop an NLP-based chatbot that can engage in conversations with users using llama2's language understanding capabilities. The chatbot could be used in various applications such as customer service or virtual assistants.
2. Sentiment Analysis for Social Media Monitoring: Create a sentiment analysis tool that uses llama2 to analyze social media posts and identify patterns and trends in user opinions and emotions. This could be useful for businesses to monitor their brand reputation and respond to customer feedback.
3. Text Summarization for News Articles: Develop a text summarization tool that uses llama2 to automatically summarize news articles, allowing readers to quickly understand the main points of a story without having to read the entire article.
4. Pers